# Lab 2: Building LangChain Agents - SOLUTIONS

**Module 2 - AI Agents and Framework Implementation**

| Duration | Difficulty | Framework | Exercises |
|----------|------------|-----------|----------|
| 90 min | Intermediate | LangChain | 4 |

## Learning Objectives

- Build custom tools for LangChain agents
- Implement a ReAct agent from scratch
- Create multi-tool agents
- Add custom callbacks for monitoring

## Setup

In [ ]:
# Install dependencies
# !pip install langchain langchain-openai python-dotenv

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_react_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool
from langchain_core.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
from pydantic import BaseModel, Field
from typing import Optional

# Set your API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your key

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

---

## Exercise 1: Create a Calculator Tool - SOLUTION

In [ ]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    Use this tool for any math calculations.
    Input should be a valid mathematical expression like '2 + 2' or '(5 * 3) - 10'.
    """
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        # Evaluate the expression
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Test the calculator tool
print(calculator.invoke("2 + 2"))
print(calculator.invoke("(10 * 5) / 2"))
print(calculator.invoke("2 ** 8"))  # Note: ** might not work with our filter
print(calculator.invoke("100 - 37"))

---

## Exercise 2: Create a Weather Tool with Structured Input - SOLUTION

In [ ]:
# Define input schema
class WeatherInput(BaseModel):
    """Input schema for weather lookup."""
    location: str = Field(description="The city name to get weather for")
    units: Optional[str] = Field(default="celsius", description="Temperature units: celsius or fahrenheit")


def get_weather(location: str, units: str = "celsius") -> str:
    """
    Get the current weather for a location.
    This is a mock implementation - in production, call a real weather API.
    """
    # Mock weather data
    mock_weather = {
        "london": {"temp_c": 15, "condition": "Cloudy"},
        "new york": {"temp_c": 22, "condition": "Sunny"},
        "tokyo": {"temp_c": 28, "condition": "Humid"},
        "paris": {"temp_c": 18, "condition": "Partly Cloudy"},
        "sydney": {"temp_c": 25, "condition": "Clear"},
    }
    
    # Normalize location name
    location_lower = location.lower().strip()
    
    if location_lower not in mock_weather:
        return f"Weather data not available for {location}"
    
    weather = mock_weather[location_lower]
    temp = weather["temp_c"]
    
    # Convert to Fahrenheit if requested
    if units.lower() == "fahrenheit":
        temp = (temp * 9/5) + 32
        unit_symbol = "°F"
    else:
        unit_symbol = "°C"
    
    return f"Weather in {location.title()}: {temp}{unit_symbol}, {weather['condition']}"


# Create the structured tool
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="weather",
    description="Get current weather for a city. Provide location and optionally units (celsius/fahrenheit).",
    args_schema=WeatherInput
)

In [ ]:
# Test the weather tool
print(weather_tool.invoke({"location": "London"}))
print(weather_tool.invoke({"location": "Tokyo", "units": "fahrenheit"}))
print(weather_tool.invoke({"location": "Paris"}))

---

## Exercise 3: Build a Multi-Tool ReAct Agent - SOLUTION

In [ ]:
# ReAct prompt template
REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(REACT_PROMPT)

In [ ]:
# Create the agent with tools
tools = [calculator, weather_tool]

# Create the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

In [ ]:
# Test the agent with various queries
queries = [
    "What is 25 * 4 + 100?",
    "What's the weather like in London?",
    "If it's 15 degrees in London and 28 degrees in Tokyo, what's the temperature difference?"
]

for query in queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print("-" * 60)
    result = agent_executor.invoke({"input": query})
    print(f"\nFinal Answer: {result['output']}")

---

## Exercise 4: Implement Custom Callbacks - SOLUTION

In [ ]:
class AgentMonitorCallback(BaseCallbackHandler):
    """Custom callback handler for monitoring agent execution."""
    
    def __init__(self):
        self.steps = []
        self.total_tokens = 0
        self.llm_calls = 0
    
    def on_llm_start(self, serialized, prompts, **kwargs):
        """Called when LLM starts processing."""
        self.llm_calls += 1
        print(f"\n[CALLBACK] LLM Call #{self.llm_calls} starting...")
    
    def on_llm_end(self, response, **kwargs):
        """Called when LLM finishes."""
        # Track token usage if available
        if hasattr(response, 'llm_output') and response.llm_output:
            token_usage = response.llm_output.get('token_usage', {})
            tokens = token_usage.get('total_tokens', 0)
            self.total_tokens += tokens
            print(f"[CALLBACK] LLM Call complete. Tokens used: {tokens}")
    
    def on_tool_start(self, serialized, input_str, **kwargs):
        """Called when a tool starts executing."""
        tool_name = serialized.get("name", "unknown")
        print(f"[CALLBACK] Tool '{tool_name}' starting with input: {input_str}")
    
    def on_tool_end(self, output, **kwargs):
        """Called when a tool finishes."""
        print(f"[CALLBACK] Tool finished with output: {output}")
    
    def on_agent_action(self, action, **kwargs):
        """Called when agent decides on an action."""
        self.steps.append({
            "type": "action",
            "tool": action.tool,
            "input": action.tool_input
        })
        print(f"[CALLBACK] Agent action: {action.tool} with input: {action.tool_input}")
    
    def on_agent_finish(self, finish, **kwargs):
        """Called when agent completes."""
        self.steps.append({
            "type": "finish",
            "output": finish.return_values.get('output', '')
        })
        print(f"[CALLBACK] Agent finished!")
    
    def get_summary(self):
        """Return execution summary."""
        return {
            "total_steps": len(self.steps),
            "llm_calls": self.llm_calls,
            "tokens_used": self.total_tokens,
            "steps": self.steps
        }

In [ ]:
# Test with callbacks
callback = AgentMonitorCallback()

# Recreate executor with callbacks
agent_executor_with_callbacks = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,  # Set to False since callbacks provide our logging
    callbacks=[callback],
    handle_parsing_errors=True
)

# Run a query
print("Running query with callback monitoring...\n")
result = agent_executor_with_callbacks.invoke({
    "input": "What is 15 * 8 and what's the weather in Paris?"
})

# Get summary
print("\n" + "="*60)
print("EXECUTION SUMMARY")
print("="*60)
summary = callback.get_summary()
print(f"Total steps: {summary['total_steps']}")
print(f"LLM calls: {summary['llm_calls']}")
print(f"Tokens used: {summary['tokens_used']}")
print(f"\nStep details:")
for i, step in enumerate(summary['steps']):
    print(f"  {i+1}. {step}")

---

## Bonus: Creating a Custom Agent from Scratch

In [ ]:
def run_custom_agent(query: str, tools: list, llm, max_steps: int = 5):
    """
    A simple custom agent implementation showing the ReAct loop.
    """
    # Build tool descriptions
    tool_descriptions = "\n".join([
        f"- {t.name}: {t.description}" for t in tools
    ])
    tool_names = [t.name for t in tools]
    tool_map = {t.name: t for t in tools}
    
    # Initial prompt
    conversation = f"""You are a helpful assistant with access to tools.

Available tools:
{tool_descriptions}

To use a tool, respond with:
TOOL: <tool_name>
INPUT: <tool_input>

When you have the final answer, respond with:
FINAL: <your answer>

Question: {query}
"""
    
    print(f"Query: {query}\n")
    
    for step in range(max_steps):
        print(f"--- Step {step + 1} ---")
        
        # Get LLM response
        response = llm.invoke(conversation).content
        print(f"LLM: {response[:200]}..." if len(response) > 200 else f"LLM: {response}")
        
        # Check for final answer
        if "FINAL:" in response:
            final_answer = response.split("FINAL:")[1].strip()
            print(f"\nFinal Answer: {final_answer}")
            return final_answer
        
        # Check for tool use
        if "TOOL:" in response and "INPUT:" in response:
            tool_name = response.split("TOOL:")[1].split("\n")[0].strip()
            tool_input = response.split("INPUT:")[1].split("\n")[0].strip()
            
            if tool_name in tool_map:
                # Execute tool
                tool_result = tool_map[tool_name].invoke(tool_input)
                print(f"Tool Result: {tool_result}")
                
                # Add to conversation
                conversation += f"\n\nAssistant: {response}\nObservation: {tool_result}\n\nContinue:"
            else:
                conversation += f"\n\nAssistant: {response}\nObservation: Tool '{tool_name}' not found.\n\nContinue:"
        else:
            conversation += f"\n\nAssistant: {response}\n\nPlease use TOOL/INPUT format or provide FINAL answer:"
    
    return "Max steps reached without final answer"


# Test custom agent
# run_custom_agent("What is 50 + 25?", [calculator, weather_tool], llm)

---

## Checkpoint

Congratulations! You've completed Lab 2. You should now understand:

- How to create custom tools with the `@tool` decorator
- How to use Pydantic for structured tool inputs
- How the ReAct pattern enables reasoning + acting
- How to monitor agent execution with callbacks

**Next:** Lab 3 - Advanced Agent Patterns